In [32]:
from langchain_groq import ChatGroq


import os

# Setting the environment variable (for testing purposes)
os.environ['TAVILY_API_KEY'] = "tvly-"

llm=ChatGroq(api_key="",model="llama-3.2-11b-vision-preview")


In [7]:

# creating own db to initialize this 


import sqlite3
from datetime import datetime

# Connect to the SQLite database (creates the file if it doesn't exist)
conn = sqlite3.connect('hotel_booking.db')
c = conn.cursor()

# Create Hotels table
c.execute('''
CREATE TABLE IF NOT EXISTS Hotels (
    hotel_id INTEGER PRIMARY KEY AUTOINCREMENT,
    hotel_name TEXT NOT NULL,
    address TEXT,
    total_rooms INTEGER,
    phone TEXT
);
''')

# Create Rooms table
c.execute('''
CREATE TABLE IF NOT EXISTS Rooms (
    room_id INTEGER PRIMARY KEY AUTOINCREMENT,
    hotel_id INTEGER,
    room_number INTEGER,
    room_type TEXT,
    price_per_day REAL,
    is_occupied BOOLEAN DEFAULT 0,
    FOREIGN KEY (hotel_id) REFERENCES Hotels(hotel_id)
);
''')

# Create Users table
c.execute('''
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    phone TEXT
);
''')

# Create Bookings table
c.execute('''
CREATE TABLE IF NOT EXISTS Bookings (
    booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
    room_id INTEGER,
    user_id INTEGER,
    check_in_date TEXT,
    check_out_date TEXT,
    FOREIGN KEY (room_id) REFERENCES Rooms(room_id),
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);
''')

# Commit the changes
conn.commit()

# Function to check room availability
def check_room_availability(hotel_id):
    c.execute('''
    SELECT * FROM Rooms
    WHERE hotel_id = ? AND is_occupied = 0;
    ''', (hotel_id,))
    available_rooms = c.fetchall()
    if available_rooms:
        print("Available rooms:", available_rooms)
    else:
        print("No available rooms in this hotel.")

# Function to book a room
def book_room(room_id, user_id, check_in_date, check_out_date):
    # Check if the room is available
    c.execute('''
    SELECT * FROM Rooms
    WHERE room_id = ? AND is_occupied = 0;
    ''', (room_id,))
    room = c.fetchone()
    
    if room:
        # Insert booking into Bookings table
        c.execute('''
        INSERT INTO Bookings (room_id, user_id, check_in_date, check_out_date)
        VALUES (?, ?, ?, ?);
        ''', (room_id, user_id, check_in_date, check_out_date))
        
        # Mark the room as occupied
        c.execute('''
        UPDATE Rooms
        SET is_occupied = 1
        WHERE room_id = ?;
        ''', (room_id,))
        
        # Commit the changes
        conn.commit()
        print(f"Room {room_id} successfully booked!")
    else:
        print("Room is already occupied or does not exist.")

# Function to calculate booking cost
def calculate_booking_cost(booking_id):
    c.execute('''
    SELECT (julianday(check_out_date) - julianday(check_in_date)) * r.price_per_day AS total_price
    FROM Bookings b
    JOIN Rooms r ON b.room_id = r.room_id
    WHERE b.booking_id = ?;
    ''', (booking_id,))
    
    total_price = c.fetchone()[0]
    print(f"Total booking cost: ${total_price:.2f}")

# Sample Data Insertion
def insert_sample_data():
    # Insert sample hotel
    c.execute('''
    INSERT INTO Hotels (hotel_name, address, total_rooms, phone)
    VALUES ('Sunset Hotel', '123 Ocean Drive', 10, '123-456-7890');
    ''')
    
    # Insert sample rooms
    c.execute('''
    INSERT INTO Rooms (hotel_id, room_number, room_type, price_per_day, is_occupied)
    VALUES (1, 101, 'Single', 100.0, 0),
           (1, 102, 'Double', 150.0, 0),
           (1, 103, 'Suite', 200.0, 0);
    ''')
    
    # Insert sample user
    c.execute('''
    INSERT INTO Users (name, email, phone)
    VALUES ('John Doe', 'john@example.com', '555-1234');
    ''')
    
    # Commit the changes
    conn.commit()

# Run sample data insertion
insert_sample_data()

# Check available rooms
check_room_availability(1)

# Book a room
book_room(1, 1, '2024-10-12', '2024-10-14')

# Calculate booking cost
calculate_booking_cost(1)

# Close the connection
conn.close()


Available rooms: [(1, 1, 101, 'Single', 100.0, 0), (2, 1, 102, 'Double', 150.0, 0), (3, 1, 103, 'Suite', 200.0, 0)]
Room 1 successfully booked!
Total booking cost: $200.00


In [8]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('hotel_booking.db')

# Set the busy timeout to 5 seconds (5000 milliseconds)
conn.execute('PRAGMA busy_timeout = 5000')

# Create a cursor object
c = conn.cursor()

# Insert rooms data into the Rooms table
try:
    c.execute('''
        INSERT INTO Rooms (hotel_id, room_number, room_type, price_per_day, is_occupied)
        VALUES (1, 104, 'Single', 100.0, 0),
               (1, 105, 'Double', 150.0, 0),
               (1, 106, 'Suite', 200.0, 0),
               (1, 107, 'Single', 100.0, 0),
               (1, 108, 'Single', 200.0, 0),
               (1, 109, 'Suite', 300.0, 0),
               (1, 110, 'Double', 150.0, 0);
    ''')
    conn.commit()  # Commit the transaction
except sqlite3.OperationalError as e:
    print("Error:", e)
finally:
    conn.close()  # Always close the connection


Above we add the db and manage it using our own code 

NOW defining the tool 

In [33]:
from langchain.tools import tool
from langchain_core.runnables import RunnableConfig

# TOOLS

In [34]:
import sqlite3
from datetime import date, datetime
from typing import Optional

import pytz
from langchain_core.runnables import RunnableConfig

db = 'hotel_booking.db'  # Replace with your actual database name


@tool
def fetch_user_hotel_information(config: RunnableConfig) -> list[dict]:
    """Fetch all hotel bookings for the user along with corresponding hotel information."""
    configuration = config.get("configurable", {})
    user_id = configuration.get("user_id", None)  # Updated to use user_id
    if not user_id:
        raise ValueError("No user ID configured.")

    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    query = """
    SELECT 
        b.booking_id, b.check_in_date, b.check_out_date, 
        h.hotel_id, h.hotel_name,  
    FROM 
        Bookings b
        JOIN hotels h ON b.hotel_id = h.hotel_id
    WHERE 
        b.user_id = ?  -- Updated to filter by user_id
    """
    cursor.execute(query, (user_id,))
    rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]
    results = [dict(zip(column_names, row)) for row in rows]

    cursor.close()
    conn.close()

    return results


@tool
def update_hotel_booking(
    booking_id: str, new_check_in_date: date, new_check_out_date: date, *, config: RunnableConfig
) -> str:
    """Update the user's hotel booking to new dates."""
    configuration = config.get("configurable", {})
    user_id = configuration.get("user_id", None)  # Updated to use user_id
    if not user_id:
        raise ValueError("No user ID configured.")

    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    cursor.execute(
        "SELECT hotel_id FROM Bookings WHERE booking_id = ?", (booking_id,)
    )
    existing_booking = cursor.fetchone()
    if not existing_booking:
        cursor.close()
        conn.close()
        return "Invalid booking ID provided."

    # Check if the user actually has this booking
    cursor.execute(
        "SELECT * FROM Bookings WHERE booking_id = ? AND user_id = ?",  # Updated to check by user_id
        (booking_id, user_id),
    )
    current_booking = cursor.fetchone()
    if not current_booking:
        cursor.close()
        conn.close()
        return f"Current signed-in user with ID {user_id} not the owner of booking {booking_id}"

    # Update the booking with new dates
    cursor.execute(
        "UPDATE Bookings SET check_in_date = ?, check_out_date = ? WHERE booking_id = ?",
        (new_check_in_date, new_check_out_date, booking_id),
    )
    conn.commit()

    cursor.close()
    conn.close()
    return "Booking successfully updated."

#
@tool
def cancel_hotel_booking(booking_id: str, *, config: RunnableConfig) -> str:
    """Cancel the user's hotel booking and remove it from the database."""
    configuration = config.get("configurable", {})
    user_id = configuration.get("user_id", None)  # Updated to use user_id
    if not user_id:
        raise ValueError("No user ID configured.")

    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    cursor.execute(
        "SELECT hotel_id FROM Bookings WHERE booking_id = ?", (booking_id,)
    )
    existing_booking = cursor.fetchone()
    if not existing_booking:
        cursor.close()
        conn.close()
        return "No existing booking found for the given booking ID."

    # Check if the user actually has this booking
    cursor.execute(
        "SELECT hotel_id FROM Bookings WHERE booking_id = ? AND user_id = ?",  # Updated to check by user_id
        (booking_id, user_id),
    )
    current_booking = cursor.fetchone()
    if not current_booking:
        cursor.close()
        conn.close()
        return f"Current signed-in user with ID {user_id} not the owner of booking {booking_id}"

    # Delete the booking from the database
    cursor.execute("DELETE FROM Bookings WHERE booking_id = ?", (booking_id,))
    conn.commit()

    cursor.close()
    conn.close()
    return "Booking successfully cancelled."











# display a a room avaialbe so we can book it 
@tool
def display_available_rooms() ->list[str]:
    """Fetch and display all available rooms from the database."""
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    query = """
    SELECT 
        room_id, room_type, price_per_night, is_occupied 
    FROM 
        Rooms 
    WHERE 
        is_occupied = 1  -- Assuming 1 means available
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]

    # Print header
    print("Available Rooms:")
    print("-" * 40)
    print(f"{' | '.join(column_names)}")
    print("-" * 40)
    res=[]
    # Print each room's details
    for row in rows:
        room_details = ' | '.join(str(value) for value in row)
        res.append(room_details)
    


    cursor.close()
    conn.close()
    return res







@tool
def create_room_booking(room_number: str, check_in_date: str, check_out_date: str, *, config: RunnableConfig) -> str:
    """Create a booking for a specific room."""
    configuration = config.get("configurable", {})
    user_id = configuration.get("user_id", None)
    
    if not user_id:
        raise ValueError("No user ID configured.")
    
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    # Check if the room is available
    cursor.execute(
        "SELECT room_id, is_occupied FROM Rooms WHERE room_number = ?",
        (room_number,)
    )
    room = cursor.fetchone()

    if not room or room[1] != 1:  # Assuming 1 means available
        cursor.close()
        conn.close()
        return f"Room {room_number} is not available for booking."

    room_id = room[0]

    # Insert booking into the Bookings table
    cursor.execute(
        "INSERT INTO Bookings (user_id, room_id, check_in_date, check_out_date) VALUES (?, ?, ?, ?)",
        (user_id, room_id, check_in_date, check_out_date)
    )
    conn.commit()

    # Update the room's availability
    cursor.execute(
        "UPDATE Rooms SET is_occupied = 0 WHERE room_id = ?",
        (room_id,)
    )
    conn.commit()

    cursor.close()
    conn.close()

    return f"Booking created successfully for Room {room_number} from {check_in_date} to {check_out_date}."





# RAG TOOL

In [35]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [36]:


'''
USe this to save local query 



'''
# text=TextLoader('p_text/text.txt')
# splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=150)
# docs=text.load()

# splitted_docs=splitter.split_documents(docs)
# embded=HuggingFaceEmbeddings()
# save_retrieve=FAISS.from_documents(splitted_docs,embded)
# save_retrieve.save_local('faiss.index')

'\nUSe this to save local query \n\n\n\n'

In [37]:
embeddings=HuggingFaceEmbeddings(),
@tool
def lookup_policy(query: str) -> str:
    """Consult the company policies to check whether certain options are permitted.
    Use this before making any flight changes performing other 'write' events."""
    retriver=FAISS.load_local('faiss.index',embeddings=embeddings,allow_dangerous_deserialization=True)
    docs = retriver.similarity_search(
    query,
    k=3)
    return docs 

/tmp/ipykernel_29357/3357703349.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings(),
/home/de/anaconda3/envs/metlife/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Tools Utilities


In [38]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.prebuilt import ToolNode

In [39]:
def handle_tool_error(state)->dict:
    error=state.get('error')
    tool_call=state["messages"][-1].tool_calls
    return{

        "messages":[ToolMessage(content=f"Error:{repr(error)} please Fix this mistake .",
                                tool_call_id=tc["id"]) for tc in tool_call]
    }

def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )

 


def _print_event(event: dict, _printed: set, max_length=5000):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)



In [40]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import AnyMessage, add_messages

In [41]:

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [42]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

In [43]:
from typing import Any


class Assistant:
    def __init__(self,runnable:Runnable) -> None:
        self.runnable=runnable


    def __call__(self, state:State,config:RunnableConfig):
        while True:
            configuration=config.get("configurable",{})
            user_id=configuration.get("user_id",None)
            state={**state ,"user_info":user_id}
            result=self.runnable.invoke(state)
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


        

Prompts with user

In [44]:
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a knowledgeable customer support assistant for a hotel booking system. "
            "Your role is to assist users with inquiries regarding room availability, booking processes, and cancellation requests. "
            "Be proactive in gathering essential information, such as check-in and check-out dates, room preferences, and the number of guests. "
            "If users have specific questions about hotel policies or services, provide clear and concise answers based on the hotel's guidelines. "
            "Guide users step-by-step through the booking process to ensure a smooth and efficient experience, addressing any issues they may encounter. "
            "If a search for available rooms or bookings returns no results, kindly encourage users to modify their criteria and search again. "
            "Your priority is to achieve user satisfaction by providing prompt, accurate, and helpful responses."
            "\n\nCurrent user:\n\n{user_info}\n"
        ),
        ("placeholder", "{messages}"),
    ]
)


In [45]:
tools_of_chatbot=[
    TavilySearchResults(max_results=1),
    lookup_policy,
    update_hotel_booking,
    create_room_booking,
    display_available_rooms,
    cancel_hotel_booking,
    fetch_user_hotel_information,
    

]

In [46]:
assiitant_runnable=primary_assistant_prompt| llm.bind_tools(tools_of_chatbot)

In [47]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition


In [48]:
builder=StateGraph(State)
builder.add_node("assistant",Assistant(assiitant_runnable))
builder.add_node("tools",create_tool_node_with_fallback(tools_of_chatbot))
builder.add_edge(START,"assistant")
builder.add_conditional_edges("assistant",tools_condition)
builder.add_edge("tools","assistant")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [30]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

KeyboardInterrupt: 

In [49]:
import uuid
q=["please book my room in the hotel it should be for single person and the dates are from 3 dec to 10dec "]
thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "user_id": 1,
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

In [50]:
_printed = set()
for question in q:
    events = graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)

================================ Human Message =================================

please book my room in the hotel it should be for single person and the dates are from 3 dec to 10dec 
================================== Ai Message ==================================

<function=create_room_booking>{"check_in_date": "2023-12-03", "check_out_date": "2023-12-10", "room_number": "single room"}<function=lookup_policy>{"query": "single person booking policy"}<function=create_room_booking>{"check_in_date": "2023-12-03", "check_out_date": "2023-12-10", "room_number": "single room"}
